# Authentication

This notebook is how we authenticate our Neo4j database to TACC! We do this using tapis, and more specifically the pods API. 
Pods are a tool which allow for simple creation of a Neo4j DBMS hosted at TACC simply through some basic commands.
At a high level, we create a Tapis object associated with an individual account. Then, a pod is created under that username, with the Neo4j template. 
This pod has an associated username and password to connect to the DBMS. A simple command reveals this information. It is also straightforward to add in other users to have
admin access to this pod. The pod has an associated bolt url, which can be connected to via a Neo4j API like py2neo in a jupyter notebook. It can also be connected to via the 
Neo4j browser. And that's it! The Neo4j database is hosted and requires authentication to TACC via Tapis!

In [13]:
import os
import py2neo
import pandas
import neo4jupyter
neo4jupyter.init_notebook_mode()
from py2neo import Graph, Node, Relationship
from py2neo import GraphService
import time
import json
import pprint as pp
import requests as r
import datetime
from getpass import getpass
from tapipy.tapis import Tapis
import pytz

def show(res):
    try:
        pp.pprint(res.json())
    except:
        pp.pprint(res.text)

<IPython.core.display.Javascript object>

In [3]:
start = time.time()

# Base URL for Tapis
base_url = "https://icicle.develop.tapis.io"
username = str(input("username"))

# Get Tapis object if it isn't already created.
try:
    t
    if t.base_url == base_url and t.username == username and t.access_token:
        print("Tapis object already exists.")
        if t.access_token.expires_at < datetime.datetime.now(pytz.utc):
            print("Existing Tapis token expired, getting new token.")
            raise
    else:
        print("Creating new Tapis object.")
        raise
except:
    try:
        t = Tapis(base_url = base_url,
                  username = username,
                  password = getpass('password'))
        t.get_tokens()
    except Exception as e:
        print(f"\nBROKEN! timeout: {time.time() - start}\n")
        raise

# V3 Headers
header_dat = {"X-Tapis-token": t.access_token.access_token,
              "Content-Type": "application/json"}

# Service URL
url = f"{base_url}/v3"                   # remote

print(time.time() - start)
print(f"base_url: {base_url}")
print(f"serv_url: {url}")

8.53785252571106
base_url: https://icicle.develop.tapis.io
serv_url: https://icicle.develop.tapis.io/v3


In [4]:
# Returns all of the pods registered to the account. 
t.pods.get_pods()

[
 creation_ts: None
 data_attached: []
 data_requests: []
 description: The pod for the REHS final KG
 environment_variables: 
 
 pod_id: finalkg
 pod_template: neo4j
 roles_inherited: []
 roles_required: []
 status: RUNNING
 status_container: 
 message: Pod is running.
 phase: Running
 start_time: 2022-07-29 20:20:36+00:00
 status_requested: ON
 update_ts: None
 url: finalkg.pods.icicle.develop.tapis.io]

In [5]:
# Used to reference the pod
pod_id = str(input("Enter a pod ID")).lower()

In [8]:
# Use neo4j to create a pod for a neo4j DBMS
pod_template = str(input("enter a pod template (enter neo4j)"))

In [9]:
# Describe the pod
pod_description = str(input("enter a pod description"))

In [10]:
# Run this to create the pod
t.pods.create_pod(pod_id=pod_id, pod_template=pod_template, description=pod_description)


creation_ts: None
data_attached: []
data_requests: []
description: sample description
environment_variables: 

pod_id: michaelraytesting
pod_template: neo4j
roles_inherited: []
roles_required: []
status: REQUESTED
status_container: 

status_requested: ON
update_ts: None
url: michaelraytesting.pods.icicle.develop.tapis.io

In [27]:
# Setting permissions 
t.pods.set_pod_permission(pod_id=pod_id, user="jackkarp", level="ADMIN")
t.pods.set_pod_permission(pod_id=pod_id, user="mkyzubr", level="ADMIN")
t.pods.set_pod_permission(pod_id=pod_id, user="archita", level="ADMIN")
t.pods.set_pod_permission(pod_id=pod_id, user="tg882699", level="ADMIN")


permissions: ['ad1616:ADMIN', 'jackkarp:ADMIN', 'mkyzubr:ADMIN', 'archita:ADMIN', 'tg882699:ADMIN']

In [6]:
# Setting username and password for authentication
username, password = t.pods.get_pod_credentials(pod_id=pod_id).user_username, t.pods.get_pod_credentials(pod_id=pod_id).user_password
print(username)
print(password)

finalkg
fUoHDk52jdbmHEuWs52lnqkyn8SFk9


In [21]:
# Testing connection to the bolt service
graph = Graph(f"bolt+ssc://{pod_id}.pods.icicle.develop.tapis.io:443", auth=(username, password), secure=True, verify=True)

In [22]:
# Further testing connection via query
query = "MATCH(n) RETURN n"
graph.run(query).to_data_frame()

,n
0,{'name': 'node 1'}
1,{'name': 'node 2'}
2,{'name': 'node 3'}
3,{'name': 'node 4'}


In [19]:
# Test visualization of test nodes
neo4jupyter.draw(graph, {"Category1" : "name", "Category2" : "name"})